In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import ScalarMappable

from tqdm import tqdm
import os
import sys
sys.path.append('../')
from PIL import Image
import cv2
import open3d as o3d

from skeletor.skeleton import Octree
from skeletor.data import loadTestDataset, loadPointCloud, plotTestDatasets, TEST_DATASETS_2D, TEST_DATASETS_3D, printTestDatasets

import robust_laplacian

from scipy.signal import convolve
from scipy.spatial import KDTree
from scipy.spatial.transform import Rotation

import scipy.sparse as sparse
import scipy.sparse.linalg as sla

from pepe.topology import spatialClusterLabels

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def contractPoints(points, referencePoints=None, pointMasses=None, attraction=1.0, contraction=0.5):
    """
    Perform Laplacian contraction on a set of points, potentially in relation to
    a reference set of points (points that should attract other points but not
    move themselves).
    """
    dim = np.shape(points)[-1]

    if hasattr(referencePoints, '__iter__'):
        allPoints = np.concatenate((points, referencePoints))
        
        # Compute the laplacian and mass matrix
        L, M = robust_laplacian.point_cloud_laplacian(allPoints)

        # We should only have positive attraction towards reference
        # points, and we should have slight negative attraction towards
        # regular points (to avoid clumping)
        if hasattr(pointMasses, '__iter__'):
            pointRepulsion = attraction * pointMasses
        else:
            pointRepulsion = attraction * np.ones(len(points))
            
        # Multiply the attraction of the reference points by a very large number so
        # they don't move from their original positions much
        referencePointAttraction = attraction * np.ones(len(referencePoints))*1e6
        pointContraction = contraction * 1e3 * np.sqrt(np.mean(M.diagonal())) * np.ones(len(points))
        referencePointContraction = contraction * 1e3 * np.sqrt(np.mean(M.diagonal())) * np.ones(len(referencePoints))

        # Define weight matrices
        WH = sparse.diags(np.concatenate((pointRepulsion, referencePointAttraction)))
        WL = sparse.diags(np.concatenate((pointContraction, referencePointContraction)))  # I * laplacian_weighting

    else:
        allPoints = points
        
        # Compute the laplacian and mass matrix
        L, M = robust_laplacian.point_cloud_laplacian(allPoints)
        
        attractionWeights = attraction * np.ones(M.shape[0])
        # This is weighted by the sqrt of the mean of the mass matrix, not really sure why, but :/
        contractionWeights = contraction * 1e3 * np.sqrt(np.mean(M.diagonal())) * np.ones(M.shape[0])

        # Define weight matrices
        WH = sparse.diags(attractionWeights)
        WL = sparse.diags(contractionWeights)  # I * laplacian_weighting

    A = sparse.vstack([L.dot(WL), WH]).tocsc()
    b = np.vstack([np.zeros((allPoints.shape[0], 3)), WH.dot(allPoints)])

    A_new = A.T @ A

    # Solve each dimension separately
    solvedAxes = [sla.spsolve(A_new, A.T @ b[:,i], permc_spec='COLAMD') for i in range(dim)]
    # If we are in 2D, just add back in the previous z dimension (no need to solve it since
    # we will throw it away eventually)
    if dim == 2:
        solvedAxes += [list(points[:,2])]
    ret = np.vstack(solvedAxes).T

    if (np.isnan(ret)).all():
        #logging.warn('Matrix is exactly singular. Stopping Contraction.')
        ret = points

    return ret[:len(points)]

In [3]:
skeletonPoints = loadPointCloud('../medial_axis_2024-10-21_LG_A_PNG_T4.0.npy', downsample=10)
referencePoints = loadPointCloud('/home/jack/Workspaces/data/point_clouds/2024-10-21_LG_A_PNG_T4.0.npy', downsample=100)

adjustedSkeletonPoints = contractPoints(skeletonPoints, referencePoints, attraction=50, contraction=0.5)

In [4]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(adjustedSkeletonPoints)
pcd.paint_uniform_color((0, 0, 0))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(skeletonPoints)
pcd2.paint_uniform_color((1, 1, 0))

o3d.visualization.draw_geometries([pcd, pcd2])

In [3]:
points = loadTestDataset('orb_web_scan', extraNoise=.001)

points = np.load('/home/jack/Workspaces/data/point_clouds/2024-10-08_LG_A_PNG_MetroInv_1.npy')
# points = np.load('/home/jack/Workspaces/data/point_clouds/latro_sheet_2024-07-11_A.npy')
dsFactor = 1
order = np.arange(points.shape[0])
np.random.shuffle(order)
points = points[order][::dsFactor,:]

print(points.shape)

octree = Octree(points, 50000, verbose=False, debug=False)
octree.plot(backend='o3d', plotPoints=False, plotBoxes=False)

print(octree.skeletonPoints.shape)

(461279, 3)
(1054, 3)


In [11]:
boxIndex = 28

skeletonPoints = [octree.boxes[boxIndex].getBoxCentroid()] + [b.getBoxCentroid() for b in octree.boxes[boxIndex].neighbors] 
skeletonPoints = np.array(skeletonPoints)

nearbyPoints = list(octree.boxes[boxIndex].points) + [p for box in octree.boxes[boxIndex].neighbors for p in box.points]
nearbyPoints = np.array(nearbyPoints)

lines = [[0,i] for i in range(1,len(skeletonPoints))]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nearbyPoints)
pcd.paint_uniform_color((1, 0, 0))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(skeletonPoints)
pcd2.paint_uniform_color((0, 1, 0))

edges = o3d.geometry.LineSet()
edges.points = o3d.utility.Vector3dVector(skeletonPoints)
edges.lines = o3d.utility.Vector2iVector(lines)

o3d.visualization.draw_geometries([pcd, pcd2, edges])

[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or binding.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed with empty geometry::LineSet.
[Open3D WARNING] [SimpleShaderForLineSet] Binding failed when preparing data.
[Open3D WARNING] [SimpleShaderForLineSet] Something is wrong in compiling or bindi

In [124]:
adjustedSkeletonPoints = contractPoints(skeletonPoints, nearbyPoints, attraction=50, contraction=0.5)

print(adjustedSkeletonPoints)
print(skeletonPoints)

skeletonPoints = [octree.boxes[boxIndex].getBoxCentroid()] + [b.getBoxCentroid() for b in octree.boxes[boxIndex].neighbors] 
skeletonPoints = np.array(skeletonPoints)

nearbyPoints = list(octree.boxes[boxIndex].points) + [p for box in octree.boxes[boxIndex].neighbors for p in box.points]
nearbyPoints = np.array(nearbyPoints)

lines = [[0,i] for i in range(1,len(skeletonPoints))]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nearbyPoints)
pcd.paint_uniform_color((1, 0, 0))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(skeletonPoints)
pcd2.paint_uniform_color((0, 1, 0))

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(adjustedSkeletonPoints)
pcd3.paint_uniform_color((0, 0, 1))


edges = o3d.geometry.LineSet()
edges.points = o3d.utility.Vector3dVector(adjustedSkeletonPoints)
edges.lines = o3d.utility.Vector2iVector(lines)

o3d.visualization.draw_geometries([pcd, pcd2, pcd3, edges])

[[427.95139518 731.56096882 571.23185808]
 [415.27154587 716.0032989  568.1600374 ]
 [421.83800731 724.73704891 569.87393408]
 [481.52140797 748.7597728  574.99288239]]
[[422.4737  747.52454 574.35657]
 [414.0849  698.32965 564.89886]
 [418.49454 726.4365  569.9075 ]
 [491.50912 748.7374  575.0701 ]]


In [7]:
newSkeletonPoints = np.copy(octree.skeletonPoints)
kdTree = KDTree(points)
boxSize = np.sqrt(np.sum(octree.boxes[0].getBoxSize()**2))/4

nearbyPoints = kdTree.query_ball_point(newSkeletonPoints, boxSize)

for i in tqdm(range(len(newSkeletonPoints))):
    
    if len(nearbyPoints[i]) <= 35:
        continue

    #conPoints = np.concatenate((newSkeletonPoints[i][None,:], points[np.array(nearbyPoints[i], dtype=np.int64)]))
    adjustedSkeletonPoints = contractPoints(newSkeletonPoints[i][None,:], points[np.array(nearbyPoints[i], dtype=np.int64)], attraction=1, contraction=0.5)

    newSkeletonPoints[i] = adjustedSkeletonPoints[0]
    
    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(points[np.array(nearbyPoints[i], dtype=np.int64)])
    # pcd.paint_uniform_color((1, 0, 0))
    
    # pcd2 = o3d.geometry.PointCloud()
    # pcd2.points = o3d.utility.Vector3dVector(adjustedSkeletonPoints)
    # pcd2.paint_uniform_color((0, 1, 0))
    
    # pcd3 = o3d.geometry.PointCloud()
    # pcd3.points = o3d.utility.Vector3dVector(newSkeletonPoints[i:i+1])
    # pcd3.paint_uniform_color((0, 0, 1))
    
    # # lines = o3d.geometry.LineSet()
    # # lines.points = o3d.utility.Vector3dVector(octree.skeletonPoints)
    # # edges = np.array(np.where(octree.skeletonAdjMat > 0), dtype=np.int64).T
    # # lines.lines = o3d.utility.Vector2iVector(edges)
    # # lines.paint_uniform_color((0, 0, 1))
    
    
    # # edges = o3d.geometry.LineSet()
    # # edges.points = o3d.utility.Vector3dVector(skeletonPoints)
    # # edges.lines = o3d.utility.Vector2iVector(lines)
    
    # o3d.visualization.draw_geometries([pcd, pcd2, pcd3])#, lines])

100%|█████████████████████████████| 1054/1054 [00:42<00:00, 24.62it/s]


In [13]:
skeletonPoints = newSkeletonPoints

# Parameters
iterations = 5
attraction = 50
contraction = 0.5
directionThreshold = 0.5
maxTurnPoints = 2

# Enumerate over all edges
adjMat = octree.skeletonAdjMat
edgePairs = np.array(np.where(adjMat > 0), dtype=np.int64).T

edgePairs = [np.sort(e) for e in edgePairs]
edgePairs = np.unique(edgePairs, axis=0)

kdTree = KDTree(points)

boxSize = np.sqrt(np.sum(octree.boxes[0].getBoxSize()**2))/2

testAllPoints = np.zeros((0,3))

for e in tqdm(edgePairs):
    edgeLength = np.sqrt(np.sum((skeletonPoints[e[0]] - skeletonPoints[e[1]])**2))
    
    # Find nearby points
    nearbyPointsIndices = kdTree.query_ball_point(skeletonPoints[[e[0], e[1]]], edgeLength)
    nearbyPointsIndices = np.unique([ind for point in nearbyPointsIndices for ind in point])

    print(len(nearbyPointsIndices))
    if len(nearbyPointsIndices) == 0:
        continue
        
    nearbyPoints = points[nearbyPointsIndices]

    # Discretize the edge
    tArr = np.linspace(0, 1, 30)
    edgePoints = skeletonPoints[e[1]] + np.multiply.outer(tArr, (skeletonPoints[e[0]] - skeletonPoints[e[1]]))

    edgeWeights = (np.abs(tArr - np.mean(tArr))) + 1
    edgeWeights /= np.max(edgeWeights)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(nearbyPoints)
    pcd.paint_uniform_color((1, 0, 0))

    pcd3 = o3d.geometry.PointCloud()
    pcd3.points = o3d.utility.Vector3dVector(edgePoints)
    pcd3.paint_uniform_color((0, 0, 1))


    o3d.visualization.draw_geometries([pcd, pcd3])#, lines])

    # Contract the edge points with the nearby points as a reference
    #conPoints = np.concatenate(())
    adjustedPoints = edgePoints
    for i in range(iterations):
        adjustedPoints = contractPoints(adjustedPoints, nearbyPoints, pointMasses=edgeWeights, attraction=attraction, contraction=contraction)
        
    adjustedEdgePoints = adjustedPoints[:len(edgePoints)]
    # Check for differences in direction between subsequent points
    directionDiff = adjustedEdgePoints[1:] - adjustedEdgePoints[:-1]
    directionDiff = np.array([d / np.sqrt(np.sum(d**2, axis=-1)) for d in directionDiff])
    directionDiffMag = np.array([np.dot(directionDiff[i], directionDiff[i+1]) for i in range(len(directionDiff)-1)])
    
    # plt.plot(directionDiffMag)
    # plt.show()

    turnPoints = np.where(directionDiffMag < directionThreshold)[0]

    if len(turnPoints) > maxTurnPoints:
        continue

    # Fit each section as a line.
    # plt.plot(directionDiffMag)
    # plt.show()

    testAllPoints = np.concatenate((testAllPoints, adjustedEdgePoints))

    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(nearbyPoints)
    # pcd.paint_uniform_color((1, 0, 0))
    
    # pcd2 = o3d.geometry.PointCloud()
    # pcd2.points = o3d.utility.Vector3dVector(adjustedPoints)
    # pcd2.paint_uniform_color((0, 1, 0))

    # pcd3 = o3d.geometry.PointCloud()
    # pcd3.points = o3d.utility.Vector3dVector(edgePoints)
    # pcd3.paint_uniform_color((0, 0, 1))

    # pcd4 = o3d.geometry.PointCloud()
    # pcd4.points = o3d.utility.Vector3dVector(adjustedEdgePoints)
    # pcd4.paint_uniform_color((1, 0, 1))

    #o3d.visualization.draw_geometries([pcd, pcd2, pcd3, pcd4])#, lines])


  0%|                                        | 0/1023 [00:00<?, ?it/s]

1587


  0%|                                | 1/1023 [00:03<55:36,  3.26s/it]

2690


  0%|                                | 2/1023 [00:05<46:00,  2.70s/it]

...

12688


  3%|▊                            | 28/1023 [01:30<1:13:14,  4.42s/it]

61976


  3%|▊                           | 28/1023 [18:43<11:05:24, 40.13s/it]


KeyboardInterrupt: 

In [239]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(testAllPoints)
pcd.paint_uniform_color((1, 0, 0))

o3d.visualization.draw_geometries([pcd])

In [240]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.paint_uniform_color((1, 0, 0))

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(octree.skeletonPoints)
pcd2.paint_uniform_color((0, 1, 0))

pcd3 = o3d.geometry.PointCloud()
pcd3.points = o3d.utility.Vector3dVector(newSkeletonPoints)
pcd3.paint_uniform_color((0, 0, 1))

lines = o3d.geometry.LineSet()
lines.points = o3d.utility.Vector3dVector(octree.skeletonPoints)
edges = np.array(np.where(octree.skeletonAdjMat > 0), dtype=np.int64).T
lines.lines = o3d.utility.Vector2iVector(edges)
lines.paint_uniform_color((0, 0, 1))


lines2 = o3d.geometry.LineSet()
lines2.points = o3d.utility.Vector3dVector(newSkeletonPoints)
edges = np.array(np.where(octree.skeletonAdjMat > 0), dtype=np.int64).T
lines2.lines = o3d.utility.Vector2iVector(edges)
lines2.paint_uniform_color((0, 1, 1))

o3d.visualization.draw_geometries([pcd, pcd2, pcd3, lines, lines2])

In [128]:
np.max(octree.skeletonPoints - newSkeletonPoints)

0.0